In [10]:
from collections import defaultdict

# セグメント木
# op: op(x,y) return x*y みたいな演算(関数)
# e: 単位元を返す関数，op(x,e()) = op(e(),x) = x になるようなe
# n: 要素数
# v: 要素の配列
class SegTree:
    def __init__(self, op, e, n, v=None):
        # 要素数
        self._n = n
        # 二項演算
        self._op = op
        # 単位元を返す関数
        self._e = e
        # セグメント木の深さ(根は0)
        self._log = (n - 1).bit_length()
        # 葉の数
        self._size = 1 << self._log
        # セグメント木(0番目の要素は使わない，1番目の要素が根)
        self._d = [self._e()] * (self._size << 1)
        if v is not None:
            for i in range(self._n):
                self._d[self._size + i] = v[i]
            for i in range(self._size - 1, 0, -1):
                self._d[i] = self._op(self._d[i << 1], self._d[i << 1 | 1])
    # p番目の要素をxに変更(pは0-indexed)
    def set(self, p, x):
        p += self._size
        self._d[p] = x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素にxを加算(pは0-indexed)
    def add(self, p, x):
        p += self._size
        self._d[p] += x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素を返す(pは0-indexed)
    def get(self, p):
        return self._d[p + self._size]
    # op(d[l], d[l+1], ..., d[r-1])を返す(l,rは0-indexedの半開区間)
    def prod(self, l, r):
        sml, smr = self._e(), self._e()
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                sml = self._op(sml, self._d[l])
                l += 1
            if r & 1:
                r -= 1
                smr = self._op(self._d[r], smr)
            l >>= 1
            r >>= 1
        return self._op(sml, smr)
    # op(d[0], d[1], ..., d[n-1])を返す
    def all_prod(self):
        return self._d[1]
    # op(d[0], d[1], ..., d[n-1]) = d[i] となる最も左のiを返す(min, maxで使う)
    def minmax_left(self):
        minmax = self._d[1]
        idx = 1
        while idx < self._size:
            if self._d[idx*2] == minmax:
                idx *= 2
            else:
                idx = idx*2+1
        return idx-self._size

def main():
    N,M,L = map(int,input().split())
    a = list(map(int,input().split()))
    b = list(map(int,input().split()))
    no = defaultdict(set)
    for _ in range(L):
        c,d = map(lambda x: int(x)-1 ,input().split())
        no[c].add(d)
    b_set = SegTree(max, lambda: -10**18, M, b)
    ans = -float('inf')
    for i in range(N):
        for n in no[i]:
            b_set.set(n, -10**18)
        if ans < a[i] + b_set.all_prod():
            ans = a[i] + b_set.all_prod()
        for n in no[i]:
            b_set.set(n, b[n])
    print(ans)


if __name__ == "__main__":
    main()

2 3 2 6
2 3 1 5
2 3 0 4
1 2 0 4
0 1 0 4
5
